# Линейная регрессия на PyTorch

In [38]:
from IPython import display
from matplotlib import pyplot as plt 
import torch
from torch.utils.data import TensorDataset, DataLoader
import random

%matplotlib inline

In [39]:
num_inputs = 2 # количество признаков (входов)
num_examples = 1000 # количество значений
true_w = torch.tensor([2, -3.4]) # истинные значения весов
true_b = 4.2 # Истинное значение смещения
features = torch.randn((num_examples, num_inputs)) # Генерируем набор значений
labels = torch.mv(features, true_w) + true_b # Рассчитываем метки
labels += torch.randn(labels.shape) # Добавляем к меткам случайный шум

## Dataset - это экземпляр класса TensorDataset

In [40]:
dataset = TensorDataset(features, labels)
print(dataset.tensors[0])
print(dataset.tensors[1])

tensor([[-0.1046, -0.1935],
        [-0.3796,  0.7185],
        [ 1.0483, -0.2294],
        ...,
        [ 0.9518, -0.6980],
        [ 1.6272,  0.9768],
        [-1.1266, -0.1063]])
tensor([ 6.0144e+00,  1.0030e+00,  5.8587e+00,  3.2540e+00,  1.0491e+01,
         4.6643e+00,  3.0665e-01,  1.2766e+00, -2.5307e+00,  1.1919e+01,
         6.1894e+00,  5.8394e+00,  7.0353e+00,  1.9950e+00,  6.0826e+00,
         4.6110e+00,  9.1841e-01,  1.7191e-01,  1.2734e+01,  9.5882e-01,
         1.3142e+00,  8.4446e+00,  2.1904e+00, -2.0785e+00,  2.1571e+00,
         1.5068e+00, -2.2967e+00,  7.1257e+00,  9.3698e+00,  5.8868e+00,
        -5.4288e+00,  8.9215e-01,  1.0183e+01,  7.3319e+00, -7.1422e+00,
         1.0334e+01,  2.5533e+00,  3.3399e+00,  5.3251e+00,  4.1084e+00,
         2.3387e+00,  2.7671e+00,  9.6284e+00,  1.6840e+00,  2.4811e+00,
        -6.7449e+00,  4.8679e+00,  2.7981e+00,  1.0895e+01,  1.4696e+00,
         4.9905e+00, -1.5558e+00,  9.4052e+00, -1.0148e+00,  8.2235e+00,
         8.5744

## DataIterator - это экземпляр класса DataLoader

In [41]:
# Randomly reading mini-batches
batch_size = 10
data_iter = DataLoader(dataset, batch_size, shuffle=True)

### Выведем один batch размером = 10

In [42]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.4808, -1.5708],
        [-0.3719, -0.1245],
        [ 0.5361, -0.6189],
        [ 0.7405,  0.2242],
        [ 0.4126,  2.7856],
        [-0.0372,  1.6485],
        [ 0.2877, -1.1896],
        [-0.0553,  0.0922],
        [-0.3796,  0.7185],
        [ 0.3739,  1.3835]]) tensor([ 9.7351,  2.4811,  7.1257,  4.1279, -5.4757, -1.5558,  8.1889,  2.5280,
         1.0030,  0.1719])


## Строим модель

### Задаем модель

In [43]:
# var_1 (Позволяет делать мнгослойные модели)
model_01 = torch.nn.Sequential(torch.nn.Linear(2, 1))
model_01

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [44]:
# var_2 (Достаточен для нашего примера)
model_02 = torch.nn.Linear(2, 1)
model_02

Linear(in_features=2, out_features=1, bias=True)

### Инициализируем веса модели

In [45]:
# # var_1
# model_01[0].weight.data = true_w.clone().detach().requires_grad_(True).reshape((1, 2))
# model_01[0].bias.data = torch.tensor([true_b], requires_grad=True)

In [46]:
# # var_2
# model_02.weight.data = true_w.clone().detach().requires_grad_(True).reshape((1, 2))
# model_02.bias.data = torch.tensor([true_b], requires_grad=True)

> По умолчанию Torch инициализирует начальные значения параметров по методу, признанному лучним на сегодняшний день

### Функция потерь

In [47]:
loss = torch.nn.MSELoss(reduction='mean')

### Создаем опримизатор

In [48]:
# var_1
trainer_01 = torch.optim.SGD(model_01.parameters(), lr=0.001)

# var_2
trainer_02 = torch.optim.SGD(model_02.parameters(), lr=0.001)

> Надо всегда следить за размерностями тензоров!

### Обучение

In [49]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer_01.zero_grad()
        l = loss(model_01(X).reshape(-1), y)
        l.backward()
        trainer_01.step()
    l = loss(model_01(features).reshape(-1), labels)
    if epoch % 5 == 0:
        print('epoch %d, loss: %f' % (epoch, l.item()),
              '|\tw', model_01[0].weight.data,
              '|\tb', model_01[0].bias.data)    

epoch 5, loss: 5.626430 |	w tensor([[ 1.1881, -2.0304]]) |	b tensor([2.7486])
epoch 10, loss: 1.564821 |	w tensor([[ 1.6468, -2.9375]]) |	b tensor([3.6669])
epoch 15, loss: 1.024135 |	w tensor([[ 1.8322, -3.2471]]) |	b tensor([4.0128])
epoch 20, loss: 0.951096 |	w tensor([[ 1.9085, -3.3528]]) |	b tensor([4.1421])
epoch 25, loss: 0.940998 |	w tensor([[ 1.9381, -3.3900]]) |	b tensor([4.1911])
epoch 30, loss: 0.939613 |	w tensor([[ 1.9494, -3.4017]]) |	b tensor([4.2097])
epoch 35, loss: 0.939381 |	w tensor([[ 1.9546, -3.4069]]) |	b tensor([4.2179])
epoch 40, loss: 0.939356 |	w tensor([[ 1.9561, -3.4073]]) |	b tensor([4.2209])
epoch 45, loss: 0.939354 |	w tensor([[ 1.9568, -3.4071]]) |	b tensor([4.2212])
epoch 50, loss: 0.939353 |	w tensor([[ 1.9576, -3.4076]]) |	b tensor([4.2214])
epoch 55, loss: 0.939351 |	w tensor([[ 1.9574, -3.4079]]) |	b tensor([4.2224])
epoch 60, loss: 0.939351 |	w tensor([[ 1.9578, -3.4082]]) |	b tensor([4.2226])
epoch 65, loss: 0.939352 |	w tensor([[ 1.9576, -3.408

In [51]:
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer_02.zero_grad()
        l = loss(model_02(X).reshape(-1), y)
        l.backward()
        trainer_02.step()
    l = loss(model_02(features).reshape(-1), labels)
    if epoch % 5 == 0:
        print('epoch %d, loss: %f' % (epoch, l.item()),
              '|\tw', model_02.weight.data,
              '|\tb', model_02.bias.data)  

epoch 5, loss: 1.538781 |	w tensor([[ 1.6442, -2.9352]]) |	b tensor([3.6953])
epoch 10, loss: 1.020771 |	w tensor([[ 1.8311, -3.2468]]) |	b tensor([4.0223])
epoch 15, loss: 0.950474 |	w tensor([[ 1.9072, -3.3534]]) |	b tensor([4.1465])
epoch 20, loss: 0.940902 |	w tensor([[ 1.9381, -3.3897]]) |	b tensor([4.1930])
epoch 25, loss: 0.939573 |	w tensor([[ 1.9489, -3.4015]]) |	b tensor([4.2121])
epoch 30, loss: 0.939387 |	w tensor([[ 1.9547, -3.4047]]) |	b tensor([4.2184])
epoch 35, loss: 0.939355 |	w tensor([[ 1.9566, -3.4071]]) |	b tensor([4.2212])
epoch 40, loss: 0.939354 |	w tensor([[ 1.9566, -3.4089]]) |	b tensor([4.2227])
epoch 45, loss: 0.939352 |	w tensor([[ 1.9573, -3.4078]]) |	b tensor([4.2231])
epoch 50, loss: 0.939353 |	w tensor([[ 1.9588, -3.4077]]) |	b tensor([4.2234])
epoch 55, loss: 0.939352 |	w tensor([[ 1.9582, -3.4083]]) |	b tensor([4.2227])
epoch 60, loss: 0.939352 |	w tensor([[ 1.9583, -3.4085]]) |	b tensor([4.2232])
epoch 65, loss: 0.939354 |	w tensor([[ 1.9580, -3.409